In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Data Transformation

## control vocabulary procedure
1. split kw by "; "
2. replace by table of human control vocab
3. replace by lemma and stem
4. node table will look like this

        id label LEMMA STEM
        1    a      a    a
        2    b      a    a
        3    c      c    c
        4    d      d    d
        5    e      e    e
5. edge table remain the same

        source target weight
            1      3      1
            2      3      1
            3      4      1
            3      5      1
            4      5      1

6. new node table will look like this

        id label LEMMA STEM
        1    a      a    a
        3    c      c    c
        4    d      d    d
        5    e      e    e


7. new edge table 0 becomes

        source target weight iloc
            a      c      1    0
            a      c      1    1
            c      d      1    2
            c      e      1    3
            d      e      1    4

8. sort_by_source_target, drop_duplicates and make weight become iloc[i+1] - iloc[i]
9. it'll becomes
        source target weight iloc
            a      c      2    0
            c      d      1    2
            c      e      1    3
            d      e      1    4
10. new edge table 1 replace label by new node table list.index
        source target weight iloc
            1      3      2    0
            3      4      1    2
            3      5      1    3
            4      5      1    4

In [18]:
import os
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from datetime import datetime

path = os.getcwd()

scient_coauthor_path = path+"\\data transformation output\\scientometrics\\co-author\\"
scient_cooccurr_path = path+"\\data transformation output\\scientometrics\\co-occurrence\\"
jasist_coauthor_path = path+"\\data transformation output\\jasist\\co-author\\"
jasist_cooccurr_path = path+"\\data transformation output\\jasist\\co-occurrence\\"

f_paths = [#scient_coauthor_path\
            scient_cooccurr_path\
           #, jasist_coauthor_path\
           , jasist_cooccurr_path\
          ]

In [21]:
def read_nodes_edges_by_transformed_data(f_path):
    nodes_f_paths, edges_f_paths, wos_f_paths = [], [], []
    times = []
    for f in os.listdir(f_path):
        if f.endswith('node.csv'):
            nodes_f_paths.append(f_path+f)
        elif f.endswith('edge.csv'):
            edges_f_paths.append(f_path+f)
        elif f.endswith('wos.csv'):
            wos_f_paths.append(f_path+f)
        times.append(f.split("_")[0])
        
    df_nodes, df_edges = [], []
    for f in nodes_f_paths:
        df_node = pd.read_csv(f)
        df_nodes.append(df_node)
    for f in edges_f_paths:
        df_edge = pd.read_csv(f)
        df_edges.append(df_edge)
        
    times = sorted(list(set(times)))
        
    return df_nodes, df_edges, times

In [ ]:
def generate_label_edge(df_edges, df_nodes):
    df_label_edges = []
    label_edge = pd.DataFrame(columns=['source', 'target', 'weight'])

    for i, df_edge in enumerate(df_edges):
        label_edge_source, label_edge_target = [], []
        label = list(df_nodes[i]['label'])
        for j in df_edge['source']:
            label_edge_source.append(label[int(j)-1])
        for j in df_edge['target']:
            label_edge_target.append(label[int(j)-1])
        label_edge['source'] = label_edge_source
        label_edge['target'] = label_edge_target
        label_edge['weight'] = df_edge['weight']

        df_label_edges.append(label_edge)
        label_edge = pd.DataFrame(columns=['source', 'target', 'weight'])
        
    return df_label_edges
# df_label_edges = generate_label_edge(df_edges, df_nodes)

In [4]:
pd.read_csv(data_path+"control vocab.csv")

,label,control vocab
0,(hardware,computer systems
1,and equipmet),computer systems
2,computer systems,computer systems
3,search behavior,search behavior
4,search behavior,search behavior
5,search strategies,search behavior
6,search strategies,search behavior
7,antarctic studies,arctic studies
8,arctic studies,arctic studies
9,growth rate for science,growth rate for scientific publication
